Setup:

In [1]:
%pip install -Uqq fastai
from fastai.vision.all import *

Note: you may need to restart the kernel to use updated packages.


Import MNIST sample that contains samples of all numbers from 0 to 9:

In [2]:
path = untar_data(URLs.MNIST)
Path.BASE_PATH = path
(path/'testing').ls()

(#10) [Path('testing/9'),Path('testing/0'),Path('testing/7'),Path('testing/6'),Path('testing/1'),Path('testing/8'),Path('testing/4'),Path('testing/3'),Path('testing/2'),Path('testing/5')]

Create dictionary `digits` and `digits_training` of all digits in MNIST sample:

In [3]:
import os
dir = os.listdir(path/'training')
print((path/'testing'/'9').ls().sorted())

[Path('testing/9/1000.png'), Path('testing/9/1005.png'), Path('testing/9/1013.png'), Path('testing/9/104.png'), Path('testing/9/1045.png'), Path('testing/9/1048.png'), Path('testing/9/105.png'), Path('testing/9/1058.png'), Path('testing/9/1063.png'), Path('testing/9/108.png'), Path('testing/9/1081.png'), Path('testing/9/1086.png'), Path('testing/9/1088.png'), Path('testing/9/1090.png'), Path('testing/9/1103.png'), Path('testing/9/1105.png'), Path('testing/9/1107.png'), Path('testing/9/113.png'), Path('testing/9/1130.png'), Path('testing/9/1152.png'), Path('testing/9/1165.png'), Path('testing/9/118.png'), Path('testing/9/1183.png'), Path('testing/9/1192.png'), Path('testing/9/12.png'), Path('testing/9/1217.png'), Path('testing/9/1228.png'), Path('testing/9/1232.png'), Path('testing/9/1247.png'), Path('testing/9/125.png'), Path('testing/9/1255.png'), Path('testing/9/1277.png'), Path('testing/9/1282.png'), Path('testing/9/1304.png'), Path('testing/9/1308.png'), Path('testing/9/1309.png'),

In [4]:
import os

dir = os.listdir(path/'training')
dir_valid = os.listdir(path/'testing')

digits = {}
digits['larger_group'] = []
digits['smaller_group'] = []
for digit in dir:
    if digit in ['9', '4', '7', '1']:
        digits['smaller_group'] += (path/'testing'/digit).ls().sorted()
    else:
        digits['larger_group'] += (path/'testing'/digit).ls().sorted()

    
digits_valid = {}
digits_valid['larger_group'] = []
digits_valid['smaller_group'] = []
for digit in dir_valid:
    if digit in ['9', '4', '7', '1']:
        digits_valid['smaller_group'] += (path/'testing'/digit).ls().sorted()
    else:
        digits_valid['larger_group'] += (path/'testing'/digit).ls().sorted()

**Organize all training data:**

Transform data in `digits` elements into tuples containing stacked tensors and the amount of images:

In [5]:
for name, data in digits.items():
    digit_tensor = [tensor(Image.open(o)) for o in data]
    stacked = torch.stack(digit_tensor).float()/255
    digits[name] = (stacked, len(data))

Create training set `train_x` containing each digit in the set:

In [6]:
train_x = torch.cat([i[0] for i in digits.values()]).view(-1, 28*28)

Create training set `train_y` containing the amount of each digit in the set:

In [7]:
train_y = tensor([0]*digits['smaller_group'][1] + [1]*digits['larger_group'][1]).unsqueeze(1)
train_x.shape,train_y.shape

(torch.Size([10000, 784]), torch.Size([10000, 1]))

Create dataset of `train_x` and `train_y`:

In [8]:
dset = list(zip(train_x,train_y))

Create a `DataLoader` from `Dataset`:

In [9]:
dl = DataLoader(dset, batch_size=1024)
xb,yb = first(dl)
xb.shape,yb.shape

(torch.Size([1024, 784]), torch.Size([1024, 1]))

**Organize all validation data:**

Transform data in `digits_valid` elements into tuples containing stacked tensors and the amount of images:

In [10]:
for digit, data in digits_valid.items():
    digit_tensor = [tensor(Image.open(o)) for o in data]
    stacked_digit = torch.stack(digit_tensor).float()/255
    digits_valid[digit] = (stacked_digit, len(data))

Create training set `valid_x` containing each digit in the set:

In [11]:
valid_x = torch.cat([i[0] for i in digits_valid.values()]).view(-1, 28*28)

Create training set `valid_y` containing the amount of each digit in the set:

In [12]:
valid_y = tensor([0]*digits['smaller_group'][1] + [1]*digits['larger_group'][1]).unsqueeze(1)
valid_x.shape,valid_y.shape

(torch.Size([10000, 784]), torch.Size([10000, 1]))

Create dataset of `valid_x` and `valid_y`:

In [13]:
valid_dset = list(zip(valid_x,valid_y))

Create a `DataLoader` from `Dataset`:

In [14]:
valid_dl = DataLoader(valid_dset, batch_size=1024)

**Create dataloader of both training and validation data:**

In [15]:
dls = DataLoaders(dl, valid_dl)

**Training the model:**

Create model to be trained:

In [16]:
from learning_functions import NEURAL_NET_STRUCTURE, mnist_loss, batch_accuracy

learn = Learner(dls, NEURAL_NET_STRUCTURE, opt_func=SGD, loss_func=mnist_loss, metrics=batch_accuracy)

Train model:

In [17]:
learn.fit(100, 1)
learn.fit(100, 5)
learn.fit(100, 10)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.618270,0.491955,0.624900,00:00
1,0.595864,0.399805,0.666200,00:00
2,0.520355,0.329408,0.674600,00:00
3,0.442360,0.295934,0.705100,00:00
4,0.397691,0.259602,0.744600,00:00
5,0.365738,0.248128,0.752900,00:00
6,0.339431,0.235614,0.765700,00:00
7,0.317785,0.223396,0.778500,00:00
8,0.298999,0.212595,0.789900,00:00
9,0.282886,0.204909,0.797300,00:00


epoch,train_loss,valid_loss,batch_accuracy,time
0,0.155313,0.271431,0.729500,00:00
1,0.172157,0.215694,0.784800,00:00
2,0.172361,0.198131,0.802400,00:00
3,0.171965,0.197931,0.802400,00:00
4,0.174866,0.184921,0.814900,00:00
5,0.176116,0.176938,0.824000,00:00
6,0.174374,0.174277,0.826800,00:00
7,0.171604,0.170823,0.829700,00:00
8,0.169143,0.170760,0.829500,00:00
9,0.168011,0.166375,0.833200,00:00


epoch,train_loss,valid_loss,batch_accuracy,time
0,0.059028,0.107336,0.894600,00:00
1,0.070518,0.135107,0.866600,00:00
2,0.067828,0.100481,0.901900,00:00
3,0.068319,0.125056,0.875900,00:00
4,0.067696,0.102161,0.900000,00:00
5,0.066654,0.096120,0.905200,00:00
6,0.065398,0.084660,0.916300,00:00
7,0.065886,0.089723,0.912000,00:00
8,0.065109,0.082970,0.919300,00:00
9,0.065033,0.081129,0.922100,00:00


Export first model:

In [19]:
torch.save(learn, "models/initial_model.pkl")